In [21]:
import argparse
import datetime
import hashlib
import os
import shutil
import sys

import argparse
import logging
import time
from os.path import exists

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from StockPredictorDMM import DMM

import pyro
import pyro.contrib.examples.polyphonic_data_loader as poly
import pyro.distributions as dist
import pyro.poutine as poutine
from pyro.distributions import TransformedDistribution
from pyro.distributions.transforms import affine_autoregressive
from pyro.infer import (
    SVI,
    JitTrace_ELBO,
    Trace_ELBO,
    TraceEnum_ELBO,
    TraceTMC_ELBO,
    config_enumerate,
)
from pyro.optim import ClippedAdam

In [22]:
from StockPredictorDMM import DMM

In [23]:
class StockDataset(torch.utils.data.Dataset):
    def __init__(self, stock, 
                isval=False,
                test_size=0.1):
        
        features = ['Open', 'High', 'Low', 'Volume']
        output = ['Close']

        X_train, X_test, y_train, y_test = train_test_split(stock[features], 
                                                            stock[output],
                                                            test_size=test_size, 
                                                            shuffle=False
                                                            )
        if isval is False:                                         
            self.X = torch.from_numpy(X_train.pct_change().dropna(how='any').to_numpy())
            self.y = torch.from_numpy(y_train.pct_change().dropna(how='any').to_numpy())

        else:
            self.X = torch.from_numpy(X_test.pct_change().dropna(how='any').to_numpy())
            self.y = torch.from_numpy(y_test.pct_change().dropna(how='any').to_numpy())

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

In [24]:
features = ['Open', 'High', 'Low', 'Volume']

In [25]:
def initTrainDt(stock):
    val_df = StockDataset(stock,
                        isval=False)

    val = DataLoader(  # An. off-the-shelf class
            val_df,
            batch_size=24,  # batching is done automatically
            num_workers=4,
        )

    return val

In [99]:
df = pd.read_csv('../../stock/AAPL.csv', parse_dates=True, index_col='Date')
trainloader = initTrainDt(df)

In [101]:
trainloader = list(enumerate(trainloader))
trainloader

[(0,
  (0, [tensor([[ 4.3174e-04,  1.5473e-03,  8.6267e-03, -2.6627e-01],
            [ 6.0423e-04,  3.0040e-03,  5.1834e-04,  5.0928e-02],
            [ 7.4189e-03,  1.1124e-02,  5.6990e-03,  4.3068e-01],
            [ 1.0019e-02,  1.0748e-02,  1.2621e-02,  5.7004e-02],
            [ 6.9521e-03, -4.1868e-04,  3.0524e-03, -2.7113e-01],
            [-2.5258e-04,  4.6072e-03,  2.5359e-03,  1.2781e-01],
            [ 1.3475e-03, -5.2530e-03, -3.2884e-03, -1.8210e-02],
            [ 1.7662e-03,  2.6823e-03,  5.0757e-03, -3.5970e-02],
            [-6.4646e-03,  5.1830e-03, -4.9659e-03,  3.1893e-01],
            [ 1.4027e-02,  2.1624e-03,  1.2604e-02, -3.1147e-01],
            [-5.0000e-03, -3.4025e-03, -2.8402e-03,  7.9463e-02],
            [ 8.7939e-03,  2.9978e-03,  3.0158e-03,  2.7349e-01],
            [-3.7360e-03,  2.9888e-03,  3.3403e-04, -3.2357e-01],
            [-3.7500e-03, -5.8770e-03, -2.2543e-03,  5.2644e-02],
            [ 7.2772e-03,  1.6653e-02,  6.5272e-03,  3.9492e-01],
  

In [46]:
x = trainloader.X
x

tensor([[ 0.0004,  0.0015,  0.0086, -0.2663],
        [ 0.0006,  0.0030,  0.0005,  0.0509],
        [ 0.0074,  0.0111,  0.0057,  0.4307],
        ...,
        [-0.0317, -0.0480, -0.0253, -0.0660],
        [ 0.0092,  0.0157,  0.0199, -0.2770],
        [ 0.0096,  0.0359,  0.0188,  0.2168]], dtype=torch.float64)

In [27]:
t = torch.from_numpy(df.to_numpy())
t.shape

torch.Size([1494, 6])

In [61]:
data = poly.load_data(poly.JSB_CHORALES)
training_seq_lengths = data["train"]["sequence_lengths"]
training_data_sequences = data["train"]["sequences"]
test_seq_lengths = data["test"]["sequence_lengths"]
test_data_sequences = data["test"]["sequences"]
val_seq_lengths = data["valid"]["sequence_lengths"]
val_data_sequences = data["valid"]["sequences"]
N_train_data = len(training_seq_lengths)
N_train_time_slices = float(torch.sum(training_seq_lengths))
N_mini_batches = int(
N_train_data / 20
    + int(N_train_data % 20 > 0)
)

In [28]:
# this function takes a torch mini-batch and reverses each sequence
# (w.r.t. the temporal axis, i.e. axis=1).
def reverse_sequences(mini_batch, seq_lengths):
    reversed_mini_batch = torch.zeros_like(mini_batch)
    for b in range(mini_batch.size(0)):
        T = seq_lengths[b]
        time_slice = torch.arange(T - 1, -1, -1, device=mini_batch.device)
        reversed_sequence = torch.index_select(mini_batch[b, :, :], 0, time_slice)
        reversed_mini_batch[b, 0:T, :] = reversed_sequence
    return reversed_mini_batch

# this function returns a 0/1 mask that can be used to mask out a mini-batch
# composed of sequences of length `seq_lengths`
def get_mini_batch_mask(mini_batch, seq_lengths):
    mask = torch.zeros(mini_batch.shape[0:2])
    for b in range(mini_batch.shape[0]):
        mask[b, 0 : seq_lengths[b]] = torch.ones(seq_lengths[b])
    return mask


# this function prepares a mini-batch for training or evaluation.
# it returns a mini-batch in forward temporal order (`mini_batch`) as
# well as a mini-batch in reverse temporal order (`mini_batch_reversed`).
# it also deals with the fact that packed sequences (which are what what we
# feed to the PyTorch rnn) need to be sorted by sequence length.
def get_mini_batch(mini_batch_indices, sequences, seq_lengths, cuda=False):
    # get the sequence lengths of the mini-batch
    seq_lengths = seq_lengths[mini_batch_indices]
    # sort the sequence lengths
    _, sorted_seq_length_indices = torch.sort(seq_lengths)
    sorted_seq_length_indices = sorted_seq_length_indices.flip(0)
    sorted_seq_lengths = seq_lengths[sorted_seq_length_indices]
    sorted_mini_batch_indices = mini_batch_indices[sorted_seq_length_indices]

    # compute the length of the longest sequence in the mini-batch
    T_max = torch.max(seq_lengths)
    # this is the sorted mini-batch
    mini_batch = sequences[sorted_mini_batch_indices, 0:T_max, :]
    # this is the sorted mini-batch in reverse temporal order
    mini_batch_reversed = reverse_sequences(mini_batch, sorted_seq_lengths)
    # get mask for mini-batch
    mini_batch_mask = get_mini_batch_mask(mini_batch, sorted_seq_lengths)

    # cuda() here because need to cuda() before packing
    if cuda:
        mini_batch = mini_batch.cuda()
        mini_batch_mask = mini_batch_mask.cuda()
        mini_batch_reversed = mini_batch_reversed.cuda()

    # do sequence packing
    mini_batch_reversed = nn.utils.rnn.pack_padded_sequence(
        mini_batch_reversed, sorted_seq_lengths, batch_first=True
    )

    return mini_batch, mini_batch_reversed, mini_batch_mask, sorted_seq_lengths

In [59]:
n_eval_samples = 1
def rep(x):
    rep_shape = torch.Size([x.size(0) * n_eval_samples]) + x.size()[1:]
    repeat_dims = [1] * len(x.size())
    repeat_dims[0] = n_eval_samples
    return (
        x.repeat(repeat_dims)
        .reshape(n_eval_samples, -1)
        .transpose(1, 0)
        .reshape(rep_shape)
)

In [96]:
test_batch,test_batch_reversed,test_batch_mask, test_seq_lengths = get_mini_batch(
        torch.arange(n_eval_samples * test_data_sequences.shape[0]),
        rep(test_data_sequences),
        test_seq_lengths
    )

training_data_sequences.shape

torch.Size([229, 129, 88])

In [97]:
n_mini = list(enumerate(trainloader))

In [32]:
dmm = DMM()
    # setup optimizer
adam_params = {
        "lr": 0.0003,
        "betas": (0.96, 0.999),
        "clip_norm": 10.0,
        "lrd": 0.99996,
        "weight_decay": 2.0,
    }
adam = ClippedAdam(adam_params)
svi = SVI(dmm.model, dmm.guide, adam, loss=Trace_ELBO())

In [33]:
data = poly.load_data(poly.JSB_CHORALES)
training_seq_lengths = data["train"]["sequence_lengths"]
training_data_sequences = data["train"]["sequences"]
test_seq_lengths = data["test"]["sequence_lengths"]
test_data_sequences = data["test"]["sequences"]
val_seq_lengths = data["valid"]["sequence_lengths"]
val_data_sequences = data["valid"]["sequences"]
N_train_data = len(training_seq_lengths)
N_train_time_slices = float(torch.sum(training_seq_lengths))
N_mini_batches = int(
        N_train_data / 20
        + int(N_train_data % 20 > 0)
    )

In [34]:
n_eval_samples = 1
def rep(x):
    rep_shape = torch.Size([x.size(0) * n_eval_samples]) + x.size()[1:]
    repeat_dims = [1] * len(x.size())
    repeat_dims[0] = n_eval_samples
    return (
        x.repeat(repeat_dims)
        .reshape(n_eval_samples, -1)
        .transpose(1, 0)
        .reshape(rep_shape)
)

In [35]:
# first dimension being the batch dimension, the second dimension being the temporal dimension, 
# and the final  
# #dimension being the features (88-dimensional in our case).

val_seq_lengths = rep(val_seq_lengths)
test_seq_lengths = rep(test_seq_lengths)

val_batch, val_batch_reversed, val_batch_mask, val_seq_lengths = poly.get_mini_batch(
                            torch.arange(n_eval_samples * val_data_sequences.shape[0]),
                            rep(val_data_sequences),
                            val_seq_lengths)

val_batch.shape[0:2]

torch.Size([76, 144])